# **OJKGPT**

This is the Jupyter Notebook for the development of the 1st version of OJKGPT for the showcase demo capabilites to wealth team program. <br>
In this development, we use several data sources for enriching the RAG system of the chatbot. 

In [8]:
import nest_asyncio
nest_asyncio.apply()

In [9]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [10]:
api_key = os.getenv("AZURE_OPENAI_KEY")
api_version = os.getenv("API_VERSION")
api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
uri = os.getenv("DATABASE_URI")

In [11]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [12]:
from utils.models_definer import ModelName

# storing or not
OCR_TRESHOLD = 0.98
TOP_K = 3
NUM_BATCH_EVAL = 10
STORE = False
DELETE = False
EVAL = False
model_name = ModelName.AZURE_OPENAI
model_name_eval = ModelName.OPENAI

## **LLM Models Define**

In [13]:
from llama_index.core import Settings
from utils.models_definer import get_llm_and_embedding

# ollama/openai/azure_openai
llm,embedding_llm = get_llm_and_embedding(model_name=model_name, api_key=api_key, api_version=api_version, api_endpoint=api_endpoint)

llm_eval = get_llm_and_embedding(model_name=model_name_eval, api_key=api_key, api_version=api_version, api_endpoint=api_endpoint)[0]


Settings.llm = llm
Settings.embed_model = embedding_llm

## **Loading Documents**

### **Document Reader and Node Parser**

In [14]:
from utils.documents_reader import read_documents
from utils.node_parser import parse_nodes

path = './data'

if STORE:
    docs = read_documents(path=path, ocr_treshold=OCR_TRESHOLD)
    nodes = parse_nodes(documents=docs, llm=llm)

## **Indexing & Storing**

In [15]:
from utils.index_store import (store_vector_index, load_vector_index)

# store
if STORE:
    vector_index = store_vector_index(nodes=nodes,embed_model=embedding_llm, delete=DELETE, uri=uri)
# load
else:
    vector_index = load_vector_index(uri=uri)

Loading the vector index completed.


## **Querying**

### **Retriever**

In [16]:
vector_retriever = vector_index.as_retriever(similarity_top_k=TOP_K)

### **Prompt**

In [17]:
from llama_index.core import PromptTemplate

qa_prompt = """\
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, \
answer the query asking about banking compliance in Indonesia. 
Answer the question based on the context information.
ALWAYS ANSWER WITH USER'S LANGUAGE.
Please provide your answer with [regulation_number](file_url) in metadata 
(if possible) in the following format:

---------------------
Answer... \n\n
Source: [regulation_number](file_url) \n
---------------------

Query: {query_str}
Answer: \
"""

qa_prompt_tmpl = PromptTemplate(qa_prompt)

### **Query Engine**

In [18]:
from llama_index.core.query_engine import RetrieverQueryEngine

vector_query_engine = RetrieverQueryEngine.from_args(retriever=vector_retriever,llm=llm,)

In [19]:
vector_query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [20]:
from utils.prompt_display import display_prompt_dict

prompts_dict = vector_query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query asking about banking compliance in Indonesia. 
Answer the question based on the context information.
ALWAYS ANSWER WITH USER'S LANGUAGE.
Please provide your answer with [regulation_number](file_url) in metadata 
(if possible) in the following format:

---------------------
Answer... 


Source: [regulation_number](file_url) 

---------------------

Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


<br><br>

In [21]:
a = "Apa judul peraturan 7/33/PBI/2005?" # Pencabutan atas Peraturan Bank Indonesia Nomor 5/17/PBI/2003 tentang Persyaratan dan Tata Cara Pelaksanaan Jaminan Pemerintah terhadap Kewajiban Pembayaran Bank Perkreditan Rakyat
b = "Kapan surat edaran No. 15/26/DPbS mulai berlaku?" # 1 Agustus 2013.
c = "Siapa nama dan jabatannya yang menandatangani surat dengan nomor 1/SEOJK.04/2013?" # NURHAIDA, kepala eksekutif pengawas pasar modal
d = "Saya ingin menyelenggarakan kegiatan pasar modal berikan saya nomor surat yang membahas mengenai hal ini!" # Peraturan Pemerintah Nomor 12 Tahun 2004
e = "Berapa persen jaminan moneter pada tanggal 20 Agustus 1958?" # 7,3%
f = "Surat edaran nomor berapa yang mengatur bank umum syariah dan unit usaha syariah?" # 15/26/DPbS
g = "Apa kepanjangan dari PAPSI?" # Pedoman Akuntansi Perbankan Syariah Indonesia
h = "apa judul peraturan nomor 112/KMK.03/2001?" # Keputusan Menteri Keuangan tentang Pemotongan Pajak Penghasil Pasal 21 atas Penghasilan berupa Uang Pesangon, Uang Tebusan Pensiun, dan Tunjangan Hari Tua atau Jaminan Hari Tua
i = "Saya ingin membuat sistem informasi lembaga jasa keuangan, berikan nomor regulasi dari peraturan yang membahas tentang manejemen risiko nya!" # 4/POJK.05/2021
j = "Apa kepanjangan dari SWDKLLJ?" # Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan
k = "Berapa nilai SWDKLLJ dari sedan?" # Rp. 140.000
l = "Apa latar belakang dari peraturan NOMOR 4/POJK.05/2021?" # dalam bentuk list
m = "Apa itu LJKNB?" # Lembaga Jasa Keuangan Non Bank
n = "Apakah KMK Nomor 462/KMK.04/1998 masih berlaku" # tidak
o = "Apa itu Uang Pesangon?" # penghasilan yang dibayarkan oleh pemberi kerja kepada karyawan dengan nama dan dalam bentuk apapun sehubungan dengan berakhirnya masa kerja atau terjadi pemutusan  hubungan kerja, termasuk uang penghargaan masa kerja dan uang  ganti kerugian
p = "Apa itu CKPN?" # Cadangan Kerugian Penurunan Nilai.
q = "Kapan, dimana, dan oleh siapa surat nomor PER- 06/BL/2012 ditetapkan?" # Surat nomor PER-06/BL/2012 ditetapkan pada tanggal 22 November 2012 di Jakarta oleh Ketua Badan Pengawas Pasar Modal dan Lembaga Keuangan.
r = "Apa kepanjangan PSAK?" # Pernyataan Standar Akuntansi Keuangan
s = "Apa itu 'shahibul maal'?" # Pemilik dana pihak ketiga

query_str = k

In [22]:
from llama_index.core.response.notebook_utils import display_response

response_vector = vector_query_engine.query(query_str)
display_response(response_vector)

**`Final Response:`** Nilai SWDKLLJ untuk sedan adalah Rp140.000,00 (seratus empat puluh ribu rupiah).

Source: [36/PMK.010/2008](https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf)

### **Chat Engine**

In [23]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

chat_store = SimpleChatStore()
memory = ChatMemoryBuffer.from_defaults(
    token_limit=10000,
    chat_store=chat_store,
    chat_store_key="user1",
)

In [24]:
# from llama_index.agent.openai import OpenAIAgent
# from llama_index.core.agent.legacy.react.base import ReActAgent
from llama_index.core.chat_engine import CondenseQuestionChatEngine

chat_engine = CondenseQuestionChatEngine.from_defaults(
    llm=llm,
    memory=memory,
    query_engine=vector_query_engine,
    verbose=True,
)

In [25]:
from llama_index.core.response.notebook_utils import display_response

response = chat_engine.chat(message=query_str)
display_response(response)

Querying with: Berapa nilai SWDKLLJ dari sedan?


**`Final Response:`** Nilai SWDKLLJ untuk sedan adalah Rp140.000,00 (seratus empat puluh ribu rupiah).

Source: [36/PMK.010/2008](https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf)

In [26]:
from llama_index.core.response.notebook_utils import display_source_node

for node in response.source_nodes:
    display_source_node(node, show_source_metadata=True)    

**Node ID:** d6fb7f53-da93-4230-b482-8ca451ec8a11<br>**Similarity:** 0.9033490419387817<br>**Text:** MENTERI KEUANGAN
REPUBLIK INDONESIA
memperoleh santunan sebesar Rp 25.000.000,00 (dua puluh lima
...<br>**Metadata:** {'file_name': 'ojk-klasifikasi_bapepam-36_pmk_010_2008-26022008-pmk_nomor_36pmk_010_tahun_2008_tentang_besar_santunan_dan_sumbangan_wajib_dana_kecelakaan_lalu_lintas_jalan_menas13_1389258036_pdf.pdf', 'title': 'Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan', 'sector': 'IKNB', 'subsector': 'Asuransi', 'regulation_type': 'Klasifikasi Bapepam', 'regulation_number': '36/PMK.010/2008', 'effective_date': '26 Februari 2008', 'file_url': 'https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf', 'questions_this_excerpt_can_answer': '1. What is the regulation number and effective date of the Peraturan Menteri Keuangan regarding Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan?\n2. What is the file URL for the document containing the Peraturan Menteri Keuangan Nomor 36/PMK.010/2008?\n3. How much is the santunan amount for individuals who suffer permanent disability due to a traffic accident?\n4. What is the specific amount of santunan for different types of vehicles, such as motorcycles, cars, and trucks, in case of accidents?\n5. What is the procedure for receiving compensation if the victim of a traffic accident does not have any heirs?', 'prev_section_summary': 'The section is about a regulation issued by the Peraturan Menteri Keuangan (Ministry of Finance) in Indonesia. The regulation, numbered 36/PMK.010/2008, pertains to the amount of compensation and mandatory contributions for traffic accident victims. It falls under the sector of IKNB (Insurance) and the subsector of Asuransi (Insurance). The effective date of the regulation is February 26, 2008. The section also provides a file URL where the full document can be accessed.', 'section_summary': 'The key topics of the section are the Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 regarding Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan. The section also mentions the effective date of the regulation, which is 26 Februari 2008. The entities mentioned in the section include the Menteri Keuangan (Ministry of Finance), korban (victims), and various types of vehicles such as sepeda motor (motorcycles), mobil ambulance (ambulance cars), and mobil penumpang (passenger cars).'}<br>

**Node ID:** f23140c3-a85b-4a16-a268-d24adc2c27ea<br>**Similarity:** 0.8999478816986084<br>**Text:** MENTERI KEUANGAN
REPUBLIK INDONESIA
-4-
 Mobil penumpang angkutan umum sampai dengan 1600 cc
g. B...<br>**Metadata:** {'file_name': 'ojk-klasifikasi_bapepam-36_pmk_010_2008-26022008-pmk_nomor_36pmk_010_tahun_2008_tentang_besar_santunan_dan_sumbangan_wajib_dana_kecelakaan_lalu_lintas_jalan_menas13_1389258036_pdf.pdf', 'title': 'Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan', 'sector': 'IKNB', 'subsector': 'Asuransi', 'regulation_type': 'Klasifikasi Bapepam', 'regulation_number': '36/PMK.010/2008', 'effective_date': '26 Februari 2008', 'file_url': 'https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf', 'questions_this_excerpt_can_answer': '1. What is the effective date of Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan?\n- The effective date of the regulation is 26 Februari 2008.\n\n2. What is the regulation number for Peraturan Menteri Keuangan tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan?\n- The regulation number is 36/PMK.010/2008.\n\n3. What is the sector and subsector of the regulation?\n- The sector is IKNB (Industri Keuangan Non-Bank) and the subsector is Asuransi (Insurance).\n\n4. What is the file URL for the document containing the regulation?\n- The file URL is https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf.\n\n5. What is the title of Peraturan Menteri Keuangan Nomor 36/PMK.010/2008?\n- The title is "Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan".', 'prev_section_summary': 'The key topics of the section are the Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 regarding Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan. The section also mentions the effective date of the regulation, which is 26 Februari 2008. The entities mentioned in the section include the Menteri Keuangan (Ministry of Finance), korban (victims), and various types of vehicles such as sepeda motor (motorcycles), mobil ambulance (ambulance cars), and mobil penumpang (passenger cars).', 'section_summary': 'The key topics of the section are the Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan, its effective date, regulation number, sector and subsector, and the file URL for the document containing the regulation. The key entities mentioned are the Menteri Keuangan (Ministry of Finance), IKNB (Industri Keuangan Non-Bank), and Asuransi (Insurance).'}<br>

**Node ID:** 33f90971-71dd-46e6-865d-0f86f6f605cc<br>**Similarity:** 0.8979687094688416<br>**Text:** REPUBLIK INDONESIA
 Penyelenggaraan Usaha Perasuransian (Lembaran Negara Republik
Indonesia Tahun...<br>**Metadata:** {'file_name': 'ojk-klasifikasi_bapepam-36_pmk_010_2008-26022008-pmk_nomor_36pmk_010_tahun_2008_tentang_besar_santunan_dan_sumbangan_wajib_dana_kecelakaan_lalu_lintas_jalan_menas13_1389258036_pdf.pdf', 'title': 'Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 tentang Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan', 'sector': 'IKNB', 'subsector': 'Asuransi', 'regulation_type': 'Klasifikasi Bapepam', 'regulation_number': '36/PMK.010/2008', 'effective_date': '26 Februari 2008', 'file_url': 'https://www.ojk.go.id/id/regulasi/Documents/Pages/PMK-Nomor-36PMK.010-Tahun-2008-tentang-Besar-Santunan-dan-Sumbangan-Wajib-Dana-Kecelakaan-Lalu-Lintas-Jalan/menas13_1389258036.pdf', 'questions_this_excerpt_can_answer': '1. What is the regulation number and effective date of the Peraturan Menteri Keuangan regarding Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan?\n2. What is the file URL for the document containing the Peraturan Menteri Keuangan Nomor 36/PMK.010/2008?\n3. What is the sector and subsector that the Peraturan Menteri Keuangan Nomor 36/PMK.010/2008 falls under?\n4. What is the title of the Peraturan Menteri Keuangan Nomor 36/PMK.010/2008?\n5. What is the regulation type of the Peraturan Menteri Keuangan Nomor 36/PMK.010/2008?', 'prev_section_summary': 'The key topics of this section are the Peraturan Menteri Keuangan Nomor 36/PMK.010/2008, which pertains to the Besar Santunan dan Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan. The section also mentions the sector and subsector of this regulation, which is IKNB (Asuransi). The section provides the regulation number (36/PMK.010/2008) and effective date (26 Februari 2008) of the Peraturan Menteri Keuangan. It also includes the file name and URL of the PDF document containing the regulation. The title of the Peraturan Menteri Keuangan is mentioned as well. The contextual information references the Undang-Undang Nomor 2 Tahun 1992 and Peraturan Pemerintah Nomor 18 Tahun 1965.', 'section_summary': 'The section is about a regulation issued by the Peraturan Menteri Keuangan (Ministry of Finance) in Indonesia. The regulation, numbered 36/PMK.010/2008, pertains to the amount of compensation and mandatory contributions for traffic accident victims. It falls under the sector of IKNB (Insurance) and the subsector of Asuransi (Insurance). The effective date of the regulation is February 26, 2008. The section also provides a file URL where the full document can be accessed.'}<br>

In [27]:
chat_store_string = chat_store.json()
loaded_chat_store = SimpleChatStore.parse_raw(chat_store_string)

## **Evaluation**

In [28]:
import os

# check directory exist
if not os.path.exists("./json_data"):
    os.makedirs("./json_data")

In [29]:
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
if EVAL:
    if STORE:
        qa_dataset = generate_question_context_pairs(
            nodes=nodes,
            llm=llm_eval,
            num_questions_per_chunk=2,
        )
        qa_dataset.save_json("./json_data/pg_eval_dataset.json")
    else:
        qa_dataset = EmbeddingQAFinetuneDataset.from_json("./json_data/pg_eval_dataset.json")

### **Retrieval Evaluation**

In [30]:
metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]

In [31]:
from utils.evaluator import get_retrieval_eval_df

if EVAL:
    vector_retrieval_eval_results = await get_retrieval_eval_df(name=f"Top-{TOP_K} Eval", metrics=metrics, retriever=vector_retriever, qa_dataset=qa_dataset)
    display(vector_retrieval_eval_results)

### **Response Evaluation**

In [32]:
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    RelevancyEvaluator,
)

if EVAL:
    # model for evaluation
    relevancy_evaluator = RelevancyEvaluator(llm=llm_eval)
    faithfullness_evaluator = FaithfulnessEvaluator(llm=llm_eval)

#### **Faithfulness Evaluator**

Evaluate Response

In [33]:
from utils.evaluator import get_response_eval_df

if EVAL:
    vector_faithfullness_eval_result = faithfullness_evaluator.evaluate_response(response=response_vector, query=query_str)
    display(get_response_eval_df(query=query_str, response=response_vector, eval_result=vector_faithfullness_eval_result))

Evaluate Source Nodes

In [34]:
from utils.evaluator import get_response_eval_sources_df

if EVAL:
    relevancy_eval_sources = get_response_eval_sources_df(query=query_str, response=response_vector, evaluator=faithfullness_evaluator)
    display(relevancy_eval_sources)

#### **Relevancy Evaluator**

Evaluate Response

In [35]:
if EVAL:
    relevancy_eval_result = relevancy_evaluator.evaluate_response(
        query=query_str, response=response_vector
    )

    display(get_response_eval_df(query=query_str, response=response_vector, eval_result=relevancy_eval_result))

Evaluate Source Nodes

In [36]:
from utils.evaluator import get_response_eval_sources_df

if EVAL:
    relevancy_eval_sources = get_response_eval_sources_df(query=query_str, response=response_vector, evaluator=relevancy_evaluator)
    display(relevancy_eval_sources)

#### **Batch Evaluator (Faithfulness, Relevancy, Correctness)**

In [37]:
from llama_index.core.evaluation import BatchEvalRunner
from utils.evaluator import get_batch_eval_results
from utils.evaluator import get_batch_eval_df

if EVAL:
    runner = BatchEvalRunner(
        {
            "faithfulness": faithfullness_evaluator, 
            "relevancy": relevancy_evaluator,
        },
        workers=8,
    )

    vector_eval_results = await get_batch_eval_results(runner=runner, qa_dataset=qa_dataset, query_engine=vector_query_engine, num_queries=NUM_BATCH_EVAL)
    display(get_batch_eval_df(vector_eval_results))